In [1]:
import numpy as np
import pandas as pd
import openpyxl
pd.set_option('display.max_rows', None)
import datetime
import os
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
# ruth

path_input = r'C:\Users\YeoRuth\OneDrive - GC International AG\dataIntegrity\data_input\\'
path_output = r'C:\Users\YeoRuth\OneDrive - GC International AG\dataIntegrity\data_output\\'

In [4]:
def dataRun(df):
    df['Sales Order Date']=pd.to_datetime(df['Sales Order Date'], format="%d-%m-%Y")
    # extract duplicates
    df1 = df[df.duplicated(subset=['Account Code','SKU Code'],keep='first')]
    # extract duplicates keep all
    new = pd.DataFrame()
    for a,s in zip(df1['Account Code'], df1['SKU Code']):
        x = df[(df['Account Code']==a) & (df['SKU Code']==s)]
        new = pd.concat([new,x])
    # create date format in new columns "dateDelta"
    new['dateDelta']=[datetime.date(i.year,i.month,i.day) for i in new['Sales Order Date']]
    # sort the dataframe
    new = new.sort_values(by=['Account Code','dateDelta'])
    # create new dataframe to store duplicates which are 30 days apart
    new2=pd.DataFrame()

    # create list of unique account codes
    accCodes = new['Account Code'].unique()

    # iterates over dataframe "new" consisting of full duplicated data
    for a in accCodes:
        new1=new[new['Account Code']==a].reset_index(drop=True)
        length = len(new1)

        if length ==2:

            d0=new1.loc[0,'dateDelta']
            d1=new1.loc[1,'dateDelta']
            delta = d1-d0
            if delta.days <= 30:
#                 print(f"< 30days,{length},{a}")
                new2=pd.concat([new2,new1])

        elif length==3:

            d0=new1.loc[0,'dateDelta']
            d1=new1.loc[1,'dateDelta']
            delta = d1-d0
            if delta.days > 30:
                new2=pd.concat([new2,new1.loc[0:1]])
            d0=new1.loc[1,'dateDelta']
            d1=new1.loc[2,'dateDelta']
            delta = d1-d0
            if delta.days > 30:
                new2=pd.concat([new2,new1.lc[1:2]])
                
    
    return new2

In [30]:
# churning out individual excel files of duplicated sales data
os.chdir(path_input)
list_input = os.listdir()
for i in list_input:
    if re.search(r'^Transfer',i):
        df = pd.read_excel(i, header=4)
        cty_list = df.Country.unique()
        for cty in cty_list:
            x = df[df.Country==cty]
            result = dataRun(x)
            result.to_excel(path_output+f'duplicated_sales_{cty}.xlsx', index=False)
            
# consolidating individual excel files of duplicated sales data into one file

os.chdir(path_output)
list_output = os.listdir()
new=pd.DataFrame()
for m in list_output:
    if re.search(r'^duplicated',m):
        df1 = pd.read_excel(m)
        df1['Sales Order Date'] = [i.strftime("%Y-%m-%d") for i in df1['Sales Order Date']]
        new = pd.concat([new,df1])
        
        # remove individual files
        os.remove(m)
        
# determining the month of report
new1 = new.copy()
new1['Sales Order Date']=pd.to_datetime(new1['Sales Order Date'])
new1['Sales Order Month']=[i.strftime("%b") for i in new1['Sales Order Date']]
mth = new1['Sales Order Month'].unique()
mth_str = mth[1]+' & '+mth[0]
new.to_excel(path_output+f'crm_duplicated_sales_{mth_str}.xlsx', index=False)
